In [1]:
import xlrd

workbook = xlrd.open_workbook('maydetail.xlsx')
worksheet = workbook.sheet_by_name('Sheet1')
num_rows = worksheet.nrows - 1
curr_row = -1

data = []

titles = []


while curr_row < num_rows:
        curr_row += 1
        value = [x.value for x in worksheet.row(curr_row) if x.value]
        if curr_row < 5:
            print(value)
        if len(value) < 4:
            continue
        if not titles:
            titles = [x.value for x in worksheet.row(curr_row) if x.value] 
            continue
        row = {}
        for i in range(len(titles)):
            row[titles[i]] = worksheet.cell_value(curr_row, i)
        data.append(row)

['Find Journal Lines - NMSS']
['Company', 'NMSS National Multiple Sclerosis Society']
['Year', '2020']
['Period', 'Oct\n\nNov\n\nDec\n\nJan\n\nFeb\n\nMar\n\nApr\n\nMay\n\nJun']
['Ledger']


In [2]:
for row in data:
    row['Debit-Credit'] = row['Ledger Debit Amount'] - row['Ledger Credit Amount']

In [3]:
#for row in data:

for row in data:
    row['Accounting Date'] = xlrd.xldate.xldate_as_datetime(row['Accounting Date'], workbook.datemode)


In [4]:
for row in data:
    row['Month'] = row['Accounting Date'].month

In [5]:
workbook2 = xlrd.open_workbook('lookups.xlsx')
worksheet2 = workbook2.sheet_by_name('Sheet1')
num_rows = worksheet2.nrows - 1
curr_row = -1

data2 = []

titles2 = []


while curr_row < num_rows:
        curr_row += 1
        if not titles2:
            titles2 = [x.value for x in worksheet2.row(curr_row) if x.value] 
            continue
        row = {}
        for i in range(len(titles2)):
            row[titles2[i]] = worksheet2.cell_value(curr_row, i)
        data2.append(row)

In [6]:
data3 = {}

for row in data2:
    data3[row['Ledger Account']]=row['In/Out']

In [7]:
for row in data:
    row['In/Out'] = data3.get(row['Ledger Account'], "")

In [10]:
print(sorted(data[2].keys()))

['Accounting Date', 'Approved by', 'Bank Account', 'Company', 'Cost Center', 'Currency', 'Customer', 'Debit-Credit', 'Employee', 'External Reference ID', 'Fund', 'Fundraising Type', 'Gift', 'In/Out', 'InKind', 'Intercompany Affiliate', 'Intercompany Initiating Company', 'Journal', 'Journal Memo', 'Journal Number', 'Journal Sequence Number', 'Ledger', 'Ledger Account', 'Ledger Credit Amount', 'Ledger Debit Amount', 'Line Memo', 'Location', 'Match ID', 'Month', 'Operational Transaction', 'Originated by', 'Posted Date', 'Program', 'Project', 'Region', 'Revenue Category', 'Source', 'Spend Category', 'State', 'Status', 'Supplier']


In [11]:
v

In [12]:
print(data[2])

{'Journal': '- NMSS National Multiple Sclerosis Society - 10/01/2019 - Deferred Rent Omaha NE', 'Journal Number': '', 'Journal Sequence Number': '', 'Journal Memo': 'Deferred Rent Omaha NE', 'Operational Transaction': '', 'Originated by': 'Julie DeSantis', 'Approved by': '', 'Company': 'NMSS National Multiple Sclerosis Society', 'Intercompany Initiating Company': '', 'Status': 'Canceled', 'Accounting Date': datetime.datetime(2019, 10, 1, 0, 0), 'Posted Date': '', 'Source': 'Recurring Journal', 'Ledger': 'Actuals', 'Currency': 'USD', 'Ledger Account': '5550:Occupancy', 'Ledger Debit Amount': 28.23, 'Ledger Credit Amount': 0.0, 'Line Memo': 'Deferred Rent Omaha NE', 'External Reference ID': '', 'Spend Category': 'Office Rent Expense', 'Revenue Category': '', 'Cost Center': 'CC005 Facilities', 'Fund': 'FD001 Unrestricted', 'Gift': '', 'Program': '', 'Project': '', 'Region': 'Mid America', 'Location': 'Omaha, NE', 'Fundraising Type': '', 'InKind': '', 'State': '', 'Bank Account': '', 'Inte

In [32]:
import re

def remove_character(c, word):
    return "".join(word.split(c))

def transform_vendor_name(name):
    name = name.lower()
    name = name.strip() #took off white space
    name = re.sub(r'[^a-zA-Z]+',"",name)
    name = remove_character('inactive', name)
    if name[-3:] == 'llc':
        name = name[:-3]
    if name[-3:] == 'inc':
        name = name[:-3]
    return name



supplierset = set()

for row in data:
    if row['Supplier']:
        #supplierset.add(remove_character(" ",row['Supplier'].lower().strip())) - This also works for making it lowercase and getting rid of white space and spaces
        check = transform_vendor_name(row['Supplier'])
        if check:
            supplierset.add(check)


In [33]:
print(supplierset)

{'accessmobility', 'cityoffrankenmuth', 'bernensconvalescentpharmacyincorporated', 'delawaredepartmentoflabor', 'grandstandsportsandmemorabilia', 'ussalabamabattleshipcommission', 'pennsylvaniastateuniversity', 'wellsfargoadvisors', 'helenagorka', 'michaelclevin', 'berlintownshippolicedepartment', 'richardfernandez', 'eurekawatercompany', 'reebokinternationalltd', 'danielemiggino', 'nancylynnyovick', 'internalgardens', 'speedprordu', 'vistagrandeaptslimitedpartnership', 'alliancebaseballomaha', 'csccorporateservicecompany', 'datavailcorporation', 'metropolitanutilitiesdistrict', 'alternativehomemakingwithaheartofswflorida', 'aztecshaffer', 'marianademarcos', 'flytheenterprises', 'cityofodessa', 'illinoischaritybureaufund', 'marinamoldavskiy', 'alwaysadvancing', 'universityofedinburgh', 'siminmashreghi', 'vividlearningsystem', 'ymcaofarlington', 'rolandlappleton', 'eastpennsborotownship', 'yourpestcontrolcompany', 'stateuniversityofnewyorkatstonybrook', 'timvaughanprintingplus', 'rachae

In [34]:
supplierdict = {}

for row in data:
    if row['Supplier']:
        #make a dictionary where the key is tranformed name and the value is the original name
        check = transform_vendor_name(row['Supplier'])
        if check:
            supplierdict[check] = row['Supplier']

In [35]:
for row in data:
    row['linememoname'] = ""
    value = row['Supplier']
    if value:
        row['linememoname'] = value
        continue
    for s in list(supplierset):
        if s in transform_vendor_name(row['Line Memo']):
            row['linememoname'] = supplierdict[s]
 

In [41]:
orig = [x for x in data if x['Supplier'] == '' and x['linememoname'] != '']


In [42]:
len(orig)

18903

In [46]:
orig[5]

{'Journal': 'Operational Journal: NMSS National Multiple Sclerosis Society - 10/04/2019',
 'Journal Number': '',
 'Journal Sequence Number': 'CUST-2020-Oct-000004',
 'Journal Memo': 'FY20 Seattle On the Move Luncheon',
 'Operational Transaction': 'Customer Invoice: CI-000006',
 'Originated by': 'Elaine Ahlers',
 'Approved by': '',
 'Company': 'NMSS National Multiple Sclerosis Society',
 'Intercompany Initiating Company': '',
 'Status': 'Posted',
 'Accounting Date': datetime.datetime(2019, 10, 4, 0, 0),
 'Posted Date': 43742.20533376157,
 'Source': 'Customer Invoice',
 'Ledger': 'Actuals',
 'Currency': 'USD',
 'Ledger Account': '4002:Event Sponsorships',
 'Ledger Debit Amount': 0.0,
 'Ledger Credit Amount': 2500.0,
 'Line Memo': 'FY20 Seattle On the Move Luncheon',
 'External Reference ID': '',
 'Spend Category': '',
 'Revenue Category': 'Sponsorships',
 'Cost Center': 'CC025 Other Special Events',
 'Fund': 'FD001 Unrestricted',
 'Gift': '',
 'Program': '',
 'Project': 'FY20 On the Move